# AutoRecruiterAI

1.	**Extracting Job Postings**: It takes a career page’s scraped text and automatically extracts relevant job information (like role, experience, skills, and description) in a structured format (JSON). This helps businesses easily collect job data from multiple websites without manual effort.

2.	**Writing Cold Emails**: Based on the extracted job details, the script generates a personalized cold email that a business development executive (named Mohan) would send to a client, offering AtliQ’s services to fulfill their job needs. The email includes relevant links from the company’s portfolio.

	Explanation in Simple Terms:

	•	**What it does**: The script uses AI to quickly find job listings from a website and create professional emails to potential clients based on those job listings.
	
	•	**Why it’s useful**: It saves time by automating the extraction of job data from career pages and writing customized emails to clients without needing to manually gather information or draft messages.

In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='your_api_key', 
    model_name="llama-3.1-70b-versatile"
)


response = llm.invoke("whats tools the most HR use to hire resources.?")
print(response.content)

Here are some of the most commonly used tools by HR professionals to hire resources:

1. **Applicant Tracking Systems (ATS)**: These systems help manage job postings, candidate applications, and resumes. Examples include:
	* Workday
	* BambooHR
	* Taleo
	* iCIMS
	* Jobvite
2. **Job Boards and Career Websites**: Popular job boards and career websites used by HR professionals include:
	* LinkedIn
	* Indeed
	* Glassdoor
	* Monster
	* CareerBuilder
3. **Social Media**: Many HR professionals use social media platforms to post job openings and connect with potential candidates:
	* LinkedIn (again, a popular choice)
	* Twitter
	* Facebook
	* Instagram
4. **Recruitment Marketing Platforms**: These platforms help HR professionals create and manage job postings, as well as track candidate engagement:
	* SmashFly
	* Phenom People
	* CareerArc
5. **Video Interviewing Tools**: These tools enable HR professionals to conduct remote interviews and assessments:
	* Zoom
	* Skype
	* Google Meet
	* HireVu

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.lever.co/acceldata/1954d7e3-46a0-42b4-b712-9a16035903a2")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Acceldata - Solutions ConsultantSolutions ConsultantBengaluru / IndiaCustomer Success – Solutions & Services /Full-Time /On-siteApply for this jobAt Acceldata, We are revolutionizing data observability in how enterprises manage and observe data by offering comprehensive solutions tailored to each organization's unique needs. Our platform integrates various technologies, enabling seamless data observability for modern enterprises.About the Role: We are looking for a Solutions Consultant to join our SNS team. As a Solutions Consultant, you will work with customers on short—to medium-term engagements to implement the data observability needs using the Acceldata Data Observability Cloud(ADOC).In this role, you will leverage your expertise in big data systems, kubernetes, and cloud-native systems to design and deliver MVPs, reference architectures, and demo applications for our customers. You will provide best-in-class implementation, working closely with customer engineering teams and our 

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
job = json_res
json_res

{'role': 'Solutions Consultant',
 'experience': ['12+ years experience in data engineering, data platforms & cloud technology projects',
  '9+ years of experience in customer-facing product implementation knowledge',
  'Experience in successfully managing multiple B2B infrastructure software development projects'],
 'skills': ['Hands-on experience with two or more common Cloud ecosystems (AWS, Azure, GCP)',
  'Expertise in at least one Cloud ecosystem',
  'Comfortable writing code in either Python or Scala',
  'Design and deployment of performant end-to-end data architectures',
  'Data management concepts like Data Quality, Data Catalog and Data Governance',
  'Cloud networking and security technologies (e.g. VPC, Private Link, Private Service Connect, TLS/SSL, SASL, Kerberos, etc.)',
  'Operating Linux systems (configure, tune, and troubleshoot both RedHat and Debian-based distributions)'],
 'description': 'We are looking for a Solutions Consultant to join our SNS team. As a Solutions

In [5]:
import pandas as pd

df = pd.read_csv("portfolio.csv")
df

,Techstack,Links
0,"React, Redux, Node.js",https://brittanychiang.com/
1,"Angular, RxJS, .NET",https://rxresu.me/
2,"Vue.js, Vuetify, Firebase",https://sarahdayan.dev/
3,"Python, Django, Postgres",https://writethedocs.org/
4,"Java, Spring Boot, Hibernate",https://martinfowler.com/
5,"Flutter, Firebase, Stripe",https://timothymiller.dev/
6,"WordPress, Elementor, PHP",https://kayleigh.tech/
7,"Magento, Laravel, MySQL",https://ahmedalsamman.com/
8,"React Native, TypeScript, Firebase",https://www.darrenmothersele.com/
9,"iOS, Swift, CoreData",https://federicoterzi.com/


In [6]:
import sqlite3
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')	

import uuid
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])
        

In [7]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://www.hashicorp.com/'},
  {'links': 'https://adamwathan.me/'}],
 [{'links': 'https://www.hashicorp.com/'},
  {'links': 'https://bilalquadri.me/'}],
 [{'links': 'https://writethedocs.org/'},
  {'links': 'https://sebastianraschka.com/'}],
 [{'links': 'https://federicoterzi.com/'},
  {'links': 'https://bilalquadri.me/'}],
 [{'links': 'https://federicoterzi.com/'},
  {'links': 'https://www.hashicorp.com/'}],
 [{'links': 'https://bilalquadri.me/'},
  {'links': 'https://www.hashicorp.com/'}],
 [{'links': 'https://bilalquadri.me/'}, {'links': 'https://anilmaurya.net/'}]]

In [8]:
job


{'role': 'Solutions Consultant',
 'experience': ['12+ years experience in data engineering, data platforms & cloud technology projects',
  '9+ years of experience in customer-facing product implementation knowledge',
  'Experience in successfully managing multiple B2B infrastructure software development projects'],
 'skills': ['Hands-on experience with two or more common Cloud ecosystems (AWS, Azure, GCP)',
  'Expertise in at least one Cloud ecosystem',
  'Comfortable writing code in either Python or Scala',
  'Design and deployment of performant end-to-end data architectures',
  'Data management concepts like Data Quality, Data Catalog and Data Governance',
  'Cloud networking and security technologies (e.g. VPC, Private Link, Private Service Connect, TLS/SSL, SASL, Kerberos, etc.)',
  'Operating Linux systems (configure, tune, and troubleshoot both RedHat and Debian-based distributions)'],
 'description': 'We are looking for a Solutions Consultant to join our SNS team. As a Solutions

In [9]:
job = json_res
job['skills']

['Hands-on experience with two or more common Cloud ecosystems (AWS, Azure, GCP)',
 'Expertise in at least one Cloud ecosystem',
 'Comfortable writing code in either Python or Scala',
 'Design and deployment of performant end-to-end data architectures',
 'Data management concepts like Data Quality, Data Catalog and Data Governance',
 'Cloud networking and security technologies (e.g. VPC, Private Link, Private Service Connect, TLS/SSL, SASL, Kerberos, etc.)',
 'Operating Linux systems (configure, tune, and troubleshoot both RedHat and Debian-based distributions)']

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are HR, a business development executive at company X , X is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Comany X 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase comany X portfolio: {link_list}
        give signature randomly with company X
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions Consultant for Seamless Data Observability Implementation

Dear Hiring Manager,

We came across your job posting for a Solutions Consultant and were impressed by the role's focus on implementing data observability needs using the Acceldata Data Observability Cloud (ADOC). Our company, X, has extensive experience in facilitating the seamless integration of business processes through automated tools, and we believe our expertise aligns perfectly with your requirements.

With over a decade of experience in empowering numerous enterprises with tailored solutions, we have developed a strong portfolio in big data systems, Kubernetes, and cloud-native systems. Our team has hands-on experience with multiple Cloud ecosystems, including AWS, Azure, and GCP, and is well-versed in designing and deploying performant end-to-end data architectures.

We'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

* Our collaboration with 